In [2]:
import tensorflow as tf

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
corpus = open('tacobellmenu.csv', "r").read()
corpus = corpus.replace('\n', ' ')
corpus

'1/2-pound beef 1/2-pound beef & potato burrito 1/2-pound beef combo burrito 1/2-pound cheesy bean & rice burrito 7 layer nachos bacon cheddar gordita crunch bacon club chalupa baja chicken taco baja taco bean burrito especial beef beefy crunch burrito beefy nachos beefy potato-rito bell beefer bellburger big beef burrito big beef taco big bell box meal big border taco big crunch taco big taste taco black jack taco burrito caesar dressing cantina double steak quesadilla cheesarito cheese cheese extreme quesadilla cheese quesadilla cheesy bean and rice burrito cheesy double beef burrito cheesy gordita crunch chicken chicken burrito chicken caesar grilled stuft burrito chicken enchilada grilled stuft burrito chicken fajitas chicken fiesta melt chicken flatbread sandwich chicken nachos bellgrande chicken quesadillas chipotle grilled stuft burrito chipotle sauce cinnamon twists club chalupa creamy jalapeÃ±o sauce crispy chicken quesadilla crunchwrap supreme crunchy red strips crunchy taco 

In [4]:
characters = sorted(list(set(corpus)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print(characters)

[' ', '"', '&', ',', '-', '/', '1', '2', '3', '5', '7', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¢', '£', '®', '±', 'Â', 'Ã']


In [5]:
X = []   # extracted sequences
Y = []   # the target: follow up character for each sequence in X
length = len(corpus)
seq_length = 100

for i in range(0, length - seq_length, 1):
    sequence = corpus[i:i + seq_length]
    label = corpus[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))

Number of extracted sequences: 3912


In [6]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [7]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_modified, Y_modified, epochs=3, batch_size=128)

Epoch 1/3
3912/3912 [==============================] - 305s 78ms/step - loss: 3.0097
Epoch 2/3
3912/3912 [==============================] - 306s 78ms/step - loss: 2.9690
Epoch 3/3
3912/3912 [==============================] - 302s 77ms/step - loss: 2.9765


In [26]:
start = 5   #random row from the X array
string_mapped = list(X[start])
full_string = [n_to_char[value] for value in string_mapped]

# generating characters
for i in range(1):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]




In [27]:
string=""
for thing in full_string:
    string += thing

string

'ound beef 1/2-pound beef & potato burrito 1/2-pound beef combo burrito 1/2-pound cheesy bean & rice  '